Датасет генерируется следующим образом: есть список пар фотографий одного человека. Для каждой пары фотографий $(A, B)$ на одну из фотографий $B$ накладывается закрытие, получается закрытая фотография $B'$, после чего для всех трех фотографий вычисляются векторы в латентном пространстве. Сохраняется исходное расстояние между фотографиями $A$ и $B$ и расстояние между $A$ и $B'$.

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageColor
from psd_tools import PSDImage
from tqdm import tqdm
import dlib
import itertools
import numpy as np
import pandas as pd
import path
import Obstruct
import os
import seaborn as sns; sns.set()
%matplotlib inline

In [ ]:
DIR = './LFWcrop/faces'
IMAGE_LIST = path.glob.glob(DIR + '/*')
model = dlib.face_recognition_model_v1("models/dlib_face_recognition_resnet_model_v1.dat")
N = 31600
THRESHOLD = 0.6

In [ ]:
def LFWcropLabelGenerator(augmentation, shape=(150, 150), verbose=False):
    lists = path.glob.glob('./LFWcrop/lists/*')
    for list_ in lists:
        if list_.split('/')[-1].split('.')[0][-4:] != 'same':
            continue
        if verbose:
            print(f"Opening {list_}...")
        with open(list_, 'r') as inf:
            for line in inf:
                triple = []
                for img in line.strip().split():
                    img = Image.open("./LFWcrop/faces/" + img + ".ppm").resize(shape)
                    triple.append(np.array(img))
                augmented = augmentation(img).convert("RGB")
                triple.append(np.array(augmented))
                triple = np.stack(triple, axis=0)
                yield triple, augmented

Генерация датасета с 10 различными закрытиями, среди которых есть серые прямоугольники. Для финальной модели не использовался.

In [ ]:
def Identity(x):
    return x

augs = [
    Obstruct.obstructEyesEyepatch, Obstruct.obstructEyesSimple, Obstruct.obstructEyesSunglasses,
    Obstruct.obstructHeadHood, Obstruct.obstructMouthBlackMask, Obstruct.obstructMouthScarf,
    Obstruct.obstructMouthSimple, Obstruct.obstructMouthWhiteMask, Obstruct.obstructNoseSimple, Identity
]

In [ ]:
tobe_train_df = {'img': [], 'initial': [], 'augmented': []}
tobe_val_df = {'img': [], 'initial': [], 'augmented': []}
tobe_test_df = {'img': [], 'initial': [], 'augmented': []}
for aug in augs:
    for idx, (triple, augmented_img) in tqdm(enumerate(LFWcropLabelGenerator(aug))):
        descs = np.array(model.compute_face_descriptor(triple))
        pair_diff = np.linalg.norm(descs[0] - descs[1])
        aug_diff = np.linalg.norm(descs[0] - descs[2])
        
        random_number = np.random.choice([0, 1, 2], p=[0.8, 0.1, 0.1])
        if random_number == 0:
            path_ = os.path.join('LFWcropObstr_train', aug.__name__ + str(idx) + '.jpeg')
            augmented_img.save(path_, format='jpeg')
            tobe_train_df['img'].append(path_)
            tobe_train_df['initial'].append(pair_diff)
            tobe_train_df['augmented'].append(aug_diff)
        elif random_number == 1:
            path_ = os.path.join('LFWcropObstr_val', aug.__name__ + str(idx) + '.jpeg')
            augmented_img.save(path_, format='jpeg')
            tobe_val_df['img'].append(path_)
            tobe_val_df['initial'].append(pair_diff)
            tobe_val_df['augmented'].append(aug_diff)
        else:
            path_ = os.path.join('LFWcropObstr_test', aug.__name__ + str(idx) + '.jpeg')
            augmented_img.save(path_, format='jpeg')
            tobe_test_df['img'].append(path_)
            tobe_test_df['initial'].append(pair_diff)
            tobe_test_df['augmented'].append(aug_diff)

31600it [07:14, 72.71it/s]
31600it [06:06, 86.26it/s]
31600it [06:20, 83.15it/s]
31600it [06:23, 82.50it/s]
31600it [06:20, 83.07it/s]
31600it [06:20, 83.06it/s]
31600it [06:07, 86.00it/s]
31600it [06:49, 77.21it/s]
31600it [06:25, 82.07it/s]
31600it [06:21, 82.72it/s]


In [ ]:
train_df = pd.DataFrame(tobe_train_df)
val_df = pd.DataFrame(tobe_val_df)
test_df = pd.DataFrame(tobe_test_df)
train_df.shape, val_df.shape, test_df.shape

((252590, 3), (31770, 3), (31640, 3))

In [ ]:
train_df.to_csv('LFWcropObstr_train.csv', sep=',', index=False)

In [ ]:
val_df.to_csv('LFWcropObstr_val.csv', sep=',', index=False)

In [ ]:
test_df.to_csv('LFWcropObstr_test.csv', sep=',', index=False)

Генерация датасета с 24 закрытиями, на котором в дальнейшем обучалась финальная модель.

In [ ]:
def Identity(x):
    return x

def Eyepatch(x):
    return Obstruct.obstructEyesEyepatch(x)

def Sunglasses(x):
    return Obstruct.obstructEyesSunglasses(x)

def BlackMask(x):
    return Obstruct.obstructMouthBlackMask(x)

def Scarf(x):
    return Obstruct.obstructMouthScarf(x)

def WhiteMask(x):
    return Obstruct.obstructMouthWhiteMask(x)

def Hood(x):
    return Obstruct.obstructHeadHood(x)

eyes_augs = [Identity, Eyepatch, Sunglasses]
mouth_augs = [Identity, BlackMask, Scarf, WhiteMask]
head_augs = [Identity, Hood]

def compose(augs):
    def plc(x):
        for aug in augs:
            x = aug(x)
        return x
    return plc

In [ ]:
tobe_train_df = {'img': [], 'initial': [], 'augmented': []}
tobe_val_df = {'img': [], 'initial': [], 'augmented': []}
tobe_test_df = {'img': [], 'initial': [], 'augmented': []}
for i, (eyes, mouth, head) in enumerate(itertools.product(eyes_augs, mouth_augs, head_augs)):
    augmentation = compose([eyes, mouth, head])
    aug_name = eyes.__name__ + mouth.__name__ + head.__name__
    for idx, (triple, augmented_img) in tqdm(enumerate(LFWcropLabelGenerator(augmentation))):
        descs = np.array(model.compute_face_descriptor(triple))
        pair_diff = np.linalg.norm(descs[0] - descs[1])
        aug_diff = np.linalg.norm(descs[0] - descs[2])
        
        random_number = np.random.choice([0, 1, 2], p=[0.8, 0.1, 0.1])
        if random_number == 0:
            path_ = os.path.join('LFWcropObstr_train', aug_name + str(idx) + '.jpeg')
            augmented_img.save(path_, format='jpeg')
            tobe_train_df['img'].append(path_)
            tobe_train_df['initial'].append(pair_diff)
            tobe_train_df['augmented'].append(aug_diff)
        elif random_number == 1:
            path_ = os.path.join('LFWcropObstr_val', aug_name + str(idx) + '.jpeg')
            augmented_img.save(path_, format='jpeg')
            tobe_val_df['img'].append(path_)
            tobe_val_df['initial'].append(pair_diff)
            tobe_val_df['augmented'].append(aug_diff)
        else:
            path_ = os.path.join('LFWcropObstr_test', aug_name + str(idx) + '.jpeg')
            augmented_img.save(path_, format='jpeg')
            tobe_test_df['img'].append(path_)
            tobe_test_df['initial'].append(pair_diff)
            tobe_test_df['augmented'].append(aug_diff)

31600it [07:02, 74.87it/s]
31600it [06:49, 77.25it/s]
31600it [06:14, 84.30it/s]
31600it [07:17, 72.26it/s]
31600it [06:55, 76.01it/s]
31600it [07:21, 80.97it/s]
31600it [06:51, 76.73it/s]
31600it [07:21, 71.52it/s]
31600it [07:06, 74.07it/s]
31600it [07:27, 70.65it/s]
31600it [07:22, 71.35it/s]
31600it [07:24, 71.07it/s]
31600it [07:10, 73.36it/s]
31600it [07:56, 66.26it/s]
31600it [07:12, 73.09it/s]
31600it [07:46, 67.76it/s]
31600it [07:01, 74.92it/s]
31600it [07:26, 70.82it/s]
31600it [07:30, 70.09it/s]
31600it [07:53, 66.68it/s]
31600it [07:50, 67.10it/s]
31600it [07:41, 68.50it/s]
31600it [07:30, 70.12it/s]
31600it [07:58, 66.03it/s]


In [ ]:
train_df = pd.DataFrame(tobe_train_df)
val_df = pd.DataFrame(tobe_val_df)
test_df = pd.DataFrame(tobe_test_df)
train_df.shape, val_df.shape, test_df.shape

((606787, 3), (75814, 3), (75799, 3))

In [ ]:
train_df.to_csv('LFWcropObstr24_train.csv', sep=',', index=False)
val_df.to_csv('LFWcropObstr24_val.csv', sep=',', index=False)
test_df.to_csv('LFWcropObstr24_test.csv', sep=',', index=False)